In [107]:
    import IPython, traitlets, jinja2, fnmatch, dataclasses, ast, textwrap, collections, html, nbconvert
    with __import__('importnb').Notebook():
        from . import tangle, doctesting
    with tangle.Pidgin():
        from . import environment

    from mimetypes import guess_type
    guess = lambda x: guess_type(x)[0]

In [92]:
    def load_ipython_extension(ip):
        ip.display_formatter = StringFormatter(**ip.display_formatter._trait_values)
    
    def unload_ipython_extension(ip):
        ip.display_formatter = IPython.core.formatters.DisplayFormatter(**ip.display_formatter._trait_values)

1. `IPython.core.formatters.DisplayFormatter`
    1. `IPython.core.formatters.IPythonDisplayFormatter`
    2. `IPython.core.formatters.MimeBundleFormatter`
    3. `IPython.core.formatters.DisplayFormatter.formatters`

Lead in with one example of a parser then build out a few more.  The inline parser is proably best.

In [93]:
    def eval_shorthand_ipython(ip, str):
        return eval(ip.input_transformer_manager.transform_cell(textwrap.indent(str, ' '*4)), ip.user_ns, ip.user_global_ns)

In [94]:
    def embed(ip, str):
        type = guess(str) or ''
        if str.startswith('http'): 
            if type.startswith('image') and not type.endswith('svg'):
                return IPython.display.Image(str)
            return IPython.display.IFrame(str, '100%', '500px')


In [95]:
    def graphviz(ip, str): 
        try: import graphviz
        except: return
        if '->' in str and not str.startswith('di'): str = 'di' + str
        return graphviz.Source(str)

In [96]:
    @dataclasses.dataclass
    class PrettyMarkdown:
        data: str
        def _repr_mimebundle_(self, include=None, exclude=None): return {'text/plain': self.data, 'text/markdown': self.data}, {}

In [111]:
    def run_inline_code(ip, text):
        text, error = text.strip('`'), None
        if not text.endswith('\\'):
            with IPython.utils.capture.capture_output() as output: result = ip.run_cell(textwrap.indent(text, ' '*4), silent=True)
            error = result.error_in_exec or result.error_before_exec
        return IPython.display.HTML(
    F"""<code style="background: var(--jp-rendermime-error-background);" title="{format_title(str(error))}">{text}</code>"""
        ) if error else IPython.display.Pretty(text)

In [128]:
    def run_doctest(ip, text):
        with IPython.utils.capture.capture_output() as out:
            # variables created in the doctest namespace are stored in a specific table.
            runner = doctesting.run_docstring_examples(text, IPython.get_ipython(), verbose=True)
        runner.failures and  IPython.get_ipython().showtraceback(
            (doctesting.DocTestException, doctesting.DocTestException(out.stdout), BaseException().__traceback__))
        return IPython.display.HTML(
F"""<pre title="{format_title(out.stdout)}"><code style="background: var(--jp-rendermime-error-background);">{text}</code></pre>"""
        ) if runner.failures else IPython.display.Pretty(text)

In [129]:
    def assign_link(ip, raw):
        text, link_title = raw.lstrip('!').lstrip('[').rstrip(')').partition(')[')
        link, title = link_title.split(' ', 1)
        ip.user_ns[link] = ip.user_ns.get(link, link)
        if text.strip(): ip.user_ns[text] = ip.user_ns.get(text, raw)

In [130]:
    def format_title(object): return '&#013;'.join(
        html.escape(nbconvert.filters.strip_ansi(str(object))).splitlines())

In [131]:
    class StringFormatterException(BaseException): ...

In [132]:
    class StringFormatter(IPython.core.formatters.DisplayFormatter):
        """The parent is the displayformatter"""        
        string_formatters = traitlets.Dict(default_value={
            ';[! ]*': eval_shorthand_ipython,
            ',[! ]*': eval_shorthand_ipython,
            'https://*' : embed,
            'http://*' : embed,
            'graph*': graphviz, 
            'digraph*': graphviz,
            '>>> *': run_doctest,
            '`*[!`]*[!\\]`': run_inline_code,
            '[*](*)': assign_link,
            '![*](*)': assign_link,})
        
        def for_pattern(self, pattern, callable=None):
            if callable: self.string_formatters[pattern] = callable
            return self.string_formatters.get(pattern, None)
        
        def match_pattern(self, object):
            value = self.parent.user_ns.get(object, object)                
            if value != object: return self.format(value)

            for type, callable in self.string_formatters.items():
                if fnmatch.fnmatch(object, type):
                    value = callable(self.parent, object)
                    if (value is not None) and (value != object): object = value; break
            else: raise StringFormatterException("No formatters found.")
            return object
            
        def format(self, object, include=None, exclude=None):
            if isinstance(object, str): 
                try: object = self.match_pattern(object)
                except StringFormatterException: object = PrettyMarkdown(object)
            return super().format(object, include=include, exclude=exclude)

In [133]:
    if __name__ == '__main__':
        unload_ipython_extension(get_ipython())
        load_ipython_extension(get_ipython())
        ip = get_ipython()

In [134]:
    ">>> assert 0"

DocTestException: Finding tests in __main__
Trying:
    assert 0
Expecting nothing
**********************************************************************
File "__main__", line ?, in __main__
Failed example:
    assert 0
Exception raised:
    Traceback (most recent call last):
      File "/Users/tonyfast/anaconda/envs/p6/lib/python3.6/doctest.py", line 1330, in __run
        compileflags, 1), test.globs)
      File "<doctest __main__[0]>", line 1, in <module>
        assert 0
    AssertionError
